In [1]:
from Hamil_search import *
from qiskit.quantum_info import SparsePauliOp, Statevector

In [2]:
n = 3
H = pauliStr2mat(n, "X2")
Ht = SparsePauliOp(["IIX"], np.array([1]))
# print(H)
v = np.array(list(range(2**n)))
print(v)
print(H @ v)
vp = v.reshape((2, 2, 2))
state = Statevector(v)
print(state.data)
state = state.evolve(Ht)
print(state.data)

[0 1 2 3 4 5 6 7]
[1. 0. 3. 2. 5. 4. 7. 6.]
[0.+0.j 1.+0.j 2.+0.j 3.+0.j 4.+0.j 5.+0.j 6.+0.j 7.+0.j]
[1.+0.j 0.+0.j 3.+0.j 2.+0.j 5.+0.j 4.+0.j 7.+0.j 6.+0.j]


In [16]:
n = 9


def getXMapper(index):
    result = []
    for i in range(2**n):
        result.append(i ^ 2 ** (n - index - 1))
    return result


Xmapper = [getXMapper(i) for i in range(n)]


def applyX(v, index):
    return v[Xmapper[index]]


def applyZ(vInput, index):
    v = np.copy(vInput)
    shift = n - index - 1
    for i in range(2**n):
        if (i >> shift) % 2 == 1:
            v[i] = -v[i]
    return v


def pauli2QiskitStr(terms, num_qubit):
    result = ["I" for _ in range(num_qubit)]
    for t in terms:
        result[t[1]] = t[0]
    return "".join(result)


def Hstr2QiskitPauliOp(Hstr, num_qubit):
    result = []
    effs = []
    for term in Hstr:
        eff, paulis = splitPaulis(term)
        effs.append(eff)
        result.append(pauli2QiskitStr(paulis, num_qubit))
    return SparsePauliOp(result, np.array(effs))


def applyH(H, num_qubit, v):
    state = Statevector(v)
    state = state.evolve(H)
    return state.data


def applyHo(Hstr, v, inverse=False):
    result = np.zeros(2**n)
    for term in Hstr:
        vcopy = np.copy(v)
        eff, paulis = splitPaulis(term)
        if inverse:
            paulis = paulis[-1::-1]
        for p in paulis:
            if p[0] == "X":
                vcopy = applyX(vcopy, p[1])
            if p[0] == "Z":
                vcopy = applyZ(vcopy, p[1])
        result += eff * vcopy
    return result

In [17]:
from scipy.sparse.linalg import eigsh, LinearOperator

Mtype = np.float64
v = np.array(list(range(2**n)))
H = [
    "Z0*Z3",
    "X6*X7",
    "X1*X2",
    "Z5*Z8",
    "-4*I0",
    "X3*X4",
    "-1*X0*X1",
    "1*Z4*Z5",
    "-1*Z1*Z2",
    "1*Z6*Z7",
    "-1*Z3*Z4",
    "X7*X8",
    "-1*X4*X5",
]
if n == 16:
    H = [
        "Z0*Z1",
        "Z2*Z3",
        "X4*X8",
        "X7*X11",
        "Z12*Z13",
        "Z14*Z15",
        "-6*I0",
        "X0*X1",
        "-1*X4*X5",
        "Z1*Z2",
        "-1*Z5*Z6",
        "X2*X3",
        "-1*X6*X7",
        "Z4*Z5",
        "-1*Z8*Z9",
        "X5*X6",
        "-1*X9*X10",
        "Z6*Z7",
        "-1*Z10*Z11",
        "X8*X9",
        "-1*X12*X13",
        "Z9*Z10",
        "-1*Z13*Z14",
        "X10*X11",
        "-1*X14*X15",
    ]
Hinit = []
for h in H:
    if h[0] == "X":
        Hinit.append(h)
    elif h[0] == "-":
        if h[3] == "X":
            Hinit[-1] = Hinit[-1] + h[2:]
print(splitPaulis(Hinit[0]))
Hindex = [[t[1] for t in splitPaulis(term)[1]] for term in Hinit]
print(Hinit)
print(Hindex)

# HP = Hstr2QiskitPauliOp(H, n)
# HM = sum([pauliExpr2Mat(n, s) for s in H])
# vp = applyH(HP, n, applyH(HP.transpose(), n, v))
# vm = HM @ HM.T @ v
# print(vp - vm)

(1, [('X', 6), ('X', 7)])
['X6*X7', 'X1*X2', 'X3*X4*X0*X1', 'X7*X8*X4*X5']
[[6, 7], [1, 2], [3, 4, 0, 1], [7, 8, 4, 5]]


In [18]:
import itertools

v0 = np.zeros(2**n)
count = 0
for i in range(0, len(Hindex) + 1):
    for term in itertools.combinations(Hindex, i):
        string = ["0" for _ in range(n)]
        for t in term:
            for index in t:
                string[index] = "1" if string[index] == "0" else "0"
        ket = "".join(string)
        v0[int(ket, 2)] = 1
        count += 1
        print(term)
        print(ket)
v0 = v0 / 2 ** (len(Hindex) / 2)
print(f"{count} kets")

()
000000000
([6, 7],)
000000110
([1, 2],)
011000000
([3, 4, 0, 1],)
110110000
([7, 8, 4, 5],)
000011011
([6, 7], [1, 2])
011000110
([6, 7], [3, 4, 0, 1])
110110110
([6, 7], [7, 8, 4, 5])
000011101
([1, 2], [3, 4, 0, 1])
101110000
([1, 2], [7, 8, 4, 5])
011011011
([3, 4, 0, 1], [7, 8, 4, 5])
110101011
([6, 7], [1, 2], [3, 4, 0, 1])
101110110
([6, 7], [1, 2], [7, 8, 4, 5])
011011101
([6, 7], [3, 4, 0, 1], [7, 8, 4, 5])
110101101
([1, 2], [3, 4, 0, 1], [7, 8, 4, 5])
101101011
([6, 7], [1, 2], [3, 4, 0, 1], [7, 8, 4, 5])
101101101
16 kets


In [19]:
# Hlar = LinearOperator(
#     shape=(2**n, 2**n),
#     matvec=lambda v: applyH(HP, n, applyH(HP.transpose(), n, v)),
#     dtype=Mtype,
# )

Hlar = LinearOperator(
    shape=(2**n, 2**n),
    matvec=lambda v: applyHo(H, applyHo(H, v, inverse=True)),
    dtype=Mtype,
)
w, ev = eigsh(Hlar, 3, which="LM")
w.sort()
print(w)
print(n)
shift = w[-1]

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
[146.35760392 168.68602475 168.68602475]
9


In [20]:
# H = ["-1*Z0*X1", "Z1*X2"]


# HM = sum([pauliExpr2Mat(n, s) for s in H])
# I = np.identity(HM.shape[0])
# Hs = HM @ HM.T - shift * I
# HL = LinearOperator(shape=HM.shape, matvec=lambda v: Hs @ v, dtype=HM.dtype)
HLT = LinearOperator(
    shape=(2**n, 2**n),
    matvec=lambda v: applyHo(H, applyHo(H, v, inverse=True)) - shift * v,
    dtype=Mtype,
)

w, ev = eigsh(HLT, 3, which="LM", v0=v0, maxiter=20000)


print(w + shift)

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
iter 101
iter 102
iter 103
iter 104
iter 105
iter 106
iter 107
iter 108
iter 109
iter 110
iter 111
iter 112
iter 113
iter 114
iter 115
iter 116
iter 117
iter 118
iter 119
iter 120
iter 121
iter 122
ite

In [ ]:
H = [
    "1*Z2*X3",
    "-1*X0*Z1",
    "1*Z3*X4",
    "-1*X1*Z2",
    "1*X0*Z4",
    "-1*X2*Z3",
    "1*X3*Z4",
    "-1*Z0*X1",
    "1*Z2*X3",
    "-1*X0*Z1",
    "1*Z3*X4",
    "-1*X1*Z2",
    "1*X0*Z4",
    "-1*X2*Z3",
    "9*X1*Z4",
    "-9*Z0*X3",
]
n = 5
HM = sum([pauliExpr2Mat(n, s) for s in H])
config = {"target": 0}
eigenvectors = getSpace(n, HM, config)
print(eigenvectors.shape[1])

4


In [ ]:
with open("text", "r") as f:
    c = f.read()
    c = c.replace("\n", "")
print(c)
with open("text", "w") as f:
    f.write(c)

Quantum computing has the potential to revolutionize various fields by solving problemsintractable for classical computers. However, developing efficient quantum programs remainschallenging due to the unique constraints of quantum systems, including noise, limited qubit con-nectivity, and hardware variability. Unlike classical programming, where high-level abstractionsand optimized compilers ease development, quantum programming still relies heavily on low-levelcircuit representations, making manual implementation complex and error-prone. Program syn-thesis, an approach that automatically generates programs satisfying given specifications, offersa promising solution by optimizing quantum circuits while minimizing human effort
